In [ ]:
Task 1:
Read the following data set: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/ 
Rename the columns as per the description from this file: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

Create a sql db from adult dataset and name it sqladb

In [1]:
import pandas as pd
from pandasql import sqldf
import sqlite3
from sqlalchemy import create_engine
import functools as func

ModuleNotFoundError: No module named 'pandasql'

In [3]:
dfAdult = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)

In [4]:
dfAdult.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
dfColumnNames = pd.read_fwf('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names', sep="\n")
dfColumnNames.columns

Index(['| This data was extracted from the census bureau database found at',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6'],
      dtype='object')

In [6]:
dfColumnNames['RequiredData'] = dfColumnNames['| This data was extracted from the census bureau database found at']

In [7]:
dfColumnNames = dfColumnNames.drop(columns={'| This data was extracted from the census bureau database found at',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6'})
columnNames = dfColumnNames.iloc[95:,]['RequiredData'].str.split(":")
columnNames = [i[0] for i in columnNames]

In [8]:
#Last column name is missing so add it manually
columnNames.append('salary')

In [9]:
columnNames = func.reduce(lambda x,y : x + ',' + y, columnNames)
columnNames

NameError: name 'func' is not defined

In [10]:
columnNames = list(columnNames.replace('-', '_').split(','))

AttributeError: 'list' object has no attribute 'replace'

In [ ]:
dfAdult.columns = columnNames

In [ ]:
dfAdult.head()

In [ ]:
#engine = create_engine('sqlite:///testDB', echo=True)
conn = sqlite3.connect('testDB')
cursor = conn.cursor()

In [ ]:
import functools as func
columnNames = func.reduce(lambda x,y : x + ' %s ,' + y, dfAdult.columns.values) + ' %s'
columnNames

In [ ]:
columnNames2 = columnNames %('INTEGER', 'TEXT', 'INTEGER', 'TEXT', 'INTEGER', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'INTEGER', 'INTEGER', 'INTEGER', 'TEXT', 'TEXT')
columnNames2

In [ ]:
'create table sqldb3(%s)' %columnNames2

In [ ]:
cursor.execute('create table sqldb3(%s)' %columnNames2)

In [ ]:
sqlInsert = """insert into sqldb3 values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"""

In [ ]:
columnNames3 = func.reduce(lambda x,y: x + ', ' + 'dfAdult.' + y, dfAdult.columns.values)
columnNames3 = 'dfAdult.' + columnNames3
columnNames3

In [ ]:
cursor.executemany(sqlInsert, zip(dfAdult.age, dfAdult.workclass, dfAdult.fnlwgt, dfAdult.education, dfAdult.education_num, dfAdult.marital_status, dfAdult.occupation, dfAdult.relationship, dfAdult.race, dfAdult.sex, dfAdult.capital_gain, dfAdult.capital_loss, dfAdult.hours_per_week, dfAdult.native_country, dfAdult.salary))

In [ ]:
conn.commit()

In [ ]:
1. Select 10 records from the adult sqladb

In [ ]:
cursor.execute('select * from sqldb3 limit 10;')
cursor.fetchall()

In [ ]:
2. Show me the average hours per week of all men who are working in private sector

In [ ]:
cursor.execute('select avg(hours_per_week) from sqldb3 where workclass=" Private";')
cursor.fetchall()

In [ ]:
3. Show me the frequency table for education, occupation and relationship, separately

In [ ]:
cursor.execute('select count(education) from sqldb3;')
print('education: ')
print(cursor.fetchall())
cursor.execute('select count(occupation) from sqldb3;')
print('occupation: ')
print(cursor.fetchall())
cursor.execute('select count(relationship) from sqldb3;')
print('relationship: ')
print(cursor.fetchall())

In [ ]:
cursor.execute('select education, count(education), count(occupation), count(relationship) from sqldb3 group by education;')
print('education, total#education, total#occupation, total#relationship')
print('-'*70)
returnedRows = cursor.fetchall()
[print(x) for x in returnedRows]

In [ ]:
4. Are there any people who are married, working in private sector and having a masters degree

In [ ]:
dfAdult.head()

In [ ]:
cursor.execute("""select count(*) from sqldb3 where marital_status like "%Married-civ-spouse"
and workclass like "%Private" and education like "%Masters";""")
cursor.fetchall()

In [ ]:
5. What is the average, minimum and maximum age group for people working in different sectors

In [ ]:
cursor.execute("""select workclass, min(age), avg(age), max(age) from sqldb3 group by workclass;""")
cursor.fetchall()

In [ ]:
6. Calculate age distribution by country

In [ ]:
cursor.execute("""select native_country, min(age), avg(age), max(age) from sqldb3 
group by native_country;""")
print('Country, minimum age, average age, maximum age')
print('-'*70)
cursor.fetchall()

In [ ]:
7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

In [ ]:
cursor.execute("""select (capital_gain - capital_loss) as Net_Capital_Gain from sqldb3
where capital_gain != 0;""")
print('Net Capital Gain for 2 rows')
print(cursor.fetchone())
print(cursor.fetchone())

In [ ]:
cursor.execute("""select native_country as Country, sum(capital_gain - capital_loss) as Net_Capital_Gain from sqldb3
where capital_gain != 0 group by native_country;""")
print('Net Capital Gain for every country')
cursor.fetchall()

In [ ]:
Task 2:
Read the following data set: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, MetaData, Table, select

In [ ]:
dfAdultData = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)

In [ ]:
columnNamesTask2 = columnNames.replace("%s", "")
columnNamesTask22 = columnNamesTask2.split("  ,")
columnNamesTask22

In [ ]:
dfAdultData.columns = columnNamesTask22

In [ ]:
dfAdultData.head(3)

In [ ]:
1. Create an sqlalchemy engine using a sample from the data set

In [ ]:
engine = create_engine('sqlite:///adultData.db', echo=True)

In [ ]:
dfAdultData.to_sql('Adult', engine)

In [ ]:
2. Write two basic update queries

In [ ]:
metadata = MetaData()

In [ ]:
user = Table('Adult', metadata,
    Column('index', Integer, primary_key=True),
    Column('age', Integer),
    Column('workclass', String),
    Column('nlwgt', Integer),
    Column('education', String),
    Column('education_num', Integer),
    Column('marital_status', String),
    Column('occupation', String),
    Column('relationship', String),
    Column('race', String),
    Column('sex', String),
    Column('capital_gain', Integer),
    Column('capital_loss', Integer),
    Column('hours_per_week', Integer),
    Column('native_country', String),
    Column('salary', String),
    extend_existing=True,
    autoload=True,
    autoload_with=engine
)

In [ ]:
upd = user.update().where(user.c.native_country==' Cambodia').values(native_country='Cambodia')

In [ ]:
engine.execute(upd)

In [ ]:
upd = user.update().where(user.c.fnlwgt==1455435).values(fnlwgt=1455435)

In [ ]:
engine.execute(upd)

In [ ]:
# One more way of update query
import sqlite3
connection = sqlite3.connect("adultData.db")
cursor = connection.cursor()
cursor.execute('update Adult set native_country="Cambodia" where native_country = "%Cambodia"')
cursor.execute('update Adult set education=trim(education)')

In [ ]:
3. Write two delete queries

In [ ]:
dele = user.delete().where(user.c.capital_gain==99999)
engine.execute(dele)

In [ ]:
dele = user.delete().where(user.c.fnlwgt==1455435)
engine.execute(dele)

In [ ]:
# One more way of delete query
cursor.execute('select distinct(workclass) from Adult where capital_gain=99999').fetchone()
cursor.execute('delete from Adult where capital_gain=99999 and workclass like "%Self-emp-inc"')
cursor.execute('select max(fnlwgt) from Adult').fetchone()
cursor.execute('delete from Adult where fnlwgt=1455435')

In [ ]:
4. Write two filter queries

In [ ]:
cursor.execute('select education as "Education", education_num as "Education_No" from Adult where education_num > 10').fetchone()

In [ ]:
cursor.execute('select avg(age) as "Age", native_country as "Country" from Adult where age > 40').fetchone()

In [ ]:
5. Write two function queries

In [ ]:
cursor.execute('select trim(marital_status), marital_status from Adult').fetchone()

In [ ]:
cursor.execute('select avg(age), max(age) from Adult').fetchone()